<a href="https://colab.research.google.com/github/gennai/2024_hackaton_Bicocca/blob/main/testYolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
brain-tumor-classification-dataset


In [17]:
import os
import cv2
driveDir = "/content/drive/My Drive/KaggleTumorsDataset/brain-tumor-classification-dataset"
train_dir = "Training"
train_dir = os.path.join(driveDir, train_dir)
print(os.listdir(driveDir))
categories = ['no_tumor', 'meningioma_tumor', 'glioma_tumor', 'pituitary_tumor']

size_threshold = (10,10)
valid_extensions=('.jpg', '.png', '.jpeg')


['Testing', 'Training']


In [ ]:
def is_image_corrupt(image_path):
    try:
        img = cv2.imread(image_path)
        if img is None:
            return True
        return False
    except:
        return True

def is_image_below_threshold(img_path):
    img = cv2.imread(image_path)
    if img.shape <= size_threshold:
        print(img.shape)
        return True
    return False


for each_category in categories:
    folder_path = os.path.join(train_dir, each_category)
    for each_file in os.listdir(folder_path):
        image_path = os.path.join(folder_path, each_file)
        if os.path.isfile(image_path) and each_file.lower().endswith(valid_extensions):
            if is_image_corrupt(image_path) or is_image_below_threshold(image_path):
                os.remove(image_path)
                print(f"Removed corrupt image: {each_file}")

In [ ]:
import matplotlib.pyplot as plt
